In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rapidfuzz
import Levenshtein

In [133]:
df1=pd.read_csv("/Users/corentinpla/Documents/Statap/transactions_A.csv")
df2=pd.read_csv("/Users/corentinpla/Documents/Statap/transactions_B.csv")

In [132]:
L1=df1["tx_id"].to_list()
L2=df2["tx_id"].to_list()

In [134]:
df_A=df1.copy()
df_B=df2.copy()

In [138]:
#Base client B
def get_base_client_B(df_B)
    df_base_client1 = df_B[['sender_first_name','sender_last_name','sender_birth_date',
                           'sender_bank_id','sender_country_code',
                           'sender_zip_code']].copy()

    df_base_client2 = df_B[['receiver_first_name','receiver_last_name','receiver_birth_date', 
                            'receiver_bank_id','receiver_country_code',
                           'receiver_zip_code']].copy()

    df_base_client1.drop_duplicates(keep = 'first', inplace=True)
    df_base_client2.drop_duplicates(keep = 'first', inplace=True)

    df_base_client1.rename(columns = {'sender_first_name': 'First_name', 'sender_last_name': 'Last_name',
                                     'sender_birth_date' : 'birth_date', ' sender_account_id' : 'account_id',
                                      'sender_bank_id': 'bank_id','sender_country_code': 'country_code',
                                      'sender_zip_code': 'zip_code'},
                           inplace=True)

    df_base_client2.rename(columns = {'receiver_first_name': 'First_name', 'receiver_last_name': 'Last_name',
                                     'receiver_birth_date' : 'birth_date',' receiver_account_id' : 'account_id',
                                      'receiver_bank_id': 'bank_id','receiver_country_code': 'country_code',
                                      'receiver_zip_code': 'zip_code'},
                           inplace=True)

    df_base_client1.sort_values(by=['First_name', 'Last_name'],inplace=True)
    df_base_client2.sort_values(by=['First_name', 'Last_name'],inplace=True)

    df_base_client_B= pd.concat([df_base_client1,df_base_client2])
    df_base_client_B.drop_duplicates(keep = 'first', inplace=True)
    df_base_client_B["zip_code"]=df_base_client_B["zip_code"].apply(lambda x:str(x))
    df_base_client_B=df_base_client_B.reset_index(drop=True)
    
    return(df_base_client_B)


In [140]:
#création de la colonne string pour la base client
def get_string_client(df_base_client_B):
    
    n=len(df_base_client_B.columns)
    df_base_client_B["s"]=df_base_client_B[df_base_client_B.columns[0]]
    for k in range  (1,n) :
        df_base_client_B["s"]=df_base_client_B["s"]+df_base_client_B[df_base_client_B.columns[k]]
    
    return(df_base_client_B)

In [147]:
#création de la colonne string pour df_A
def get_string(df_A):
    Reciever=['receiver_first_name','receiver_last_name','receiver_birth_date','receiver_bank_id','receiver_country_code','receiver_zip_code']
    Sender=['sender_first_name','sender_last_name','sender_birth_date','sender_bank_id','sender_country_code','sender_zip_code']

    df_A["sender_zip_code"]=df_A["sender_zip_code"].apply(lambda x:str(x))
    df_A["receiver_zip_code"]=df_A["receiver_zip_code"].apply(lambda x:str(x))

    n=len(Reciever)

    df_A["s_reciever"]=df_A[Reciever[0]]
    df_A["s_sender"]=df_A[Sender[0]]

    for k in range(1,n):
        df_A["s_reciever"]=df_A["s_reciever"]+df_A[Reciever[k]]
        df_A["s_sender"]=df_A["s_sender"]+df_A[Sender[k]]
    
    return(df_A)

In [148]:
#prend pour argument les df_A et df_base_client_B et retourne une liste de sous dataset de pivot fixé
#Qu'est ce qu'ils entendent par groupby
def get_pivot(piv_A,piv_client_B,df_A,df_base_client_B):
    L_df=[]
    
    L_piv=df_A[piv_A].drop_duplicates()

        
    a=0
    for x in  (L_piv):
        df_A_piv=df_A[df_A[piv_A]==x]
        df_base_client_B_piv=df_base_client_B[df_base_client_B[piv_client_B]==x]
        L_df.append([df_A_piv,df_base_client_B_piv])
        a+=1
    return (L_df)

In [143]:
#side=s_reciever ou s_sender
def get_distance_levenstein(side,df_A,df_base_client_B,rowA,rowB):
    sA=df_A.at[rowA,side]
    sB=df_base_client_B.at[rowB,"s"]
    return(Levenshtein.distance(sA,sB))

In [102]:
def get_min_levenstein(side,df_A,df_base_client_B,rowA):
    min =get_distance_levenstein(side,df_A,df_base_client_B,rowA,df_base_client_B.index[0])
    ind=df_base_client_B.index[0]
    for y in df_base_client_B.index :
        dist =get_distance_levenstein(side,df_A,df_base_client_B,rowA,y)
        if dist <min :
            
            min=dist 
            ind=y
        
    return([ind,min])

In [149]:
#correction du sender sans pivot 
# df_A, df_base_client_B sont les deux dataframes
def corrige_sender(df_A,df_base_client_B):
        
    for x in df_A.index:
        print(x)
        lev =get_min_levenstein("s_sender",df_A,df_base_client_B,x)[0]
        df_A.at[x,"sender_first_name"]=df_base_client_B.at[lev,"First_name"]
        df_A.at[x,"sender_last_name"]=df_base_client_B.at[lev,"Last_name"]
        df_A.at[x,"sender_birth_date"]=df_base_client_B.at[lev,"birth_date"]
        df_A.at[x,"sender_bank_id"]=df_base_client_B.at[lev,"bank_id"]
        df_A.at[x,"sender_country_code"]=df_base_client_B.at[lev,"country_code"]
        df_A.at[x,"sender_zip_code"]=df_base_client_B.at[lev,"zip_code"]
        
        
    return(df_A)

def corrige_receiver(df_A,df_base_client_B):
        
    for x in df_A.index:
        print(x)
        lev =get_min_levenstein("s_reciever",df_A,df_base_client_B,x)[0]
        df_A.at[x,"receiver_first_name"]=df_base_client_B.at[lev,"First_name"]
        df_A.at[x,"receiver_last_name"]=df_base_client_B.at[lev,"Last_name"]
        df_A.at[x,"receiver_birth_date"]=df_base_client_B.at[lev,"birth_date"]
        df_A.at[x,"receiver_bank_id"]=df_base_client_B.at[lev,"bank_id"]
        df_A.at[x,"receiver_country_code"]=df_base_client_B.at[lev,"country_code"]
        df_A.at[x,"receiver_zip_code"]=df_base_client_B.at[lev,"zip_code"]
        
        
    return(df_A)


In [150]:
#correction du sender avec pivot
#df_A et df_base_client_B sont des sub dataframes comme présents dans L_pivot_sender
#prendre pivot sender -> corriger sender -> concatener -> prendre pivot receiver -> corriger receiver -> concaterner
def corrige_sender_pivot(L_pivot_sender):
    for couple in L_pivot_sender :
        df_A=couple[0]
        df_base_client_B=couple[1]

        
        for x in df_A.index:
            lev =get_min_levenstein("s_sender",df_A,df_base_client_B,x)[0]
            if lev[1]<4:
                df_A.at[x,"sender_first_name"]=df_base_client_B.at[lev,"First_name"]
                df_A.at[x,"sender_last_name"]=df_base_client_B.at[lev,"Last_name"]
                df_A.at[x,"sender_birth_date"]=df_base_client_B.at[lev,"birth_date"]
                df_A.at[x,"sender_bank_id"]=df_base_client_B.at[lev,"bank_id"]
                df_A.at[x,"sender_country_code"]=df_base_client_B.at[lev,"country_code"]
                df_A.at[x,"sender_zip_code"]=df_base_client_B.at[lev,"zip_code"]
        
        
    return(L_pivot_sender)

def corrige_receiver_pivot(L_pivot_receiver):
    for couple in L_pivot_receiver :
        df_A=couple[0]
        df_base_client_B=couple[1]

        
        for x in df_A.index:
            lev =get_min_levenstein("s_reciever",df_A,df_base_client_B,x)[0]
            if lev[1]<4:
                df_A.at[x,"receiver_first_name"]=df_base_client_B.at[lev,"First_name"]
                df_A.at[x,"receiver_last_name"]=df_base_client_B.at[lev,"Last_name"]
                df_A.at[x,"receiver_birth_date"]=df_base_client_B.at[lev,"birth_date"]
                df_A.at[x,"receiver_bank_id"]=df_base_client_B.at[lev,"bank_id"]
                df_A.at[x,"receiver_country_code"]=df_base_client_B.at[lev,"country_code"]
                df_A.at[x,"receiver_zip_code"]=df_base_client_B.at[lev,"zip_code"]

        
    return(L_pivot_receiver)

In [151]:
def concatenate(L_pivot):
    L=[]
    for x in L_pivot:
        L.append(x[0])
    
    return(pd.concat(L))
        
        

In [145]:
def correction_pivot(df_A,df_base_client_B):
    L_pivot_sender=get_pivot("sender_birth_date",'birth_date',df_A,df_base_client_B)
    print("1/4")
    L_pivot_sender_correct= corrige_sender_pivot(L_pivot_sender)
    print("2/4")
    df_A=concatenate(L_pivot_sender_correct)
    
    L_pivot_receiver=get_pivot("receiver_birth_date",'birth_date',df_A,df_base_client_B)
    print("3/4")
    L_pivot_receiver_correct= corrige_receiver_pivot(L_pivot_receiver)
    print("4/4")
    
    df_A=concatenate(L_pivot_receiver_correct)
    
    return(df_A)
    
    